In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load dataset
dataset = pd.read_csv('Mumbai_LSTM_10.csv')
data = dataset.values
data = data.astype('float32')

In [3]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [4]:
# Split data into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [5]:
# Create function to create LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [6]:
# Reshape input data to be 3-dimensional
X_train = np.reshape(train, (train.shape[0], 1, train.shape[1]))
X_test = np.reshape(test, (test.shape[0], 1, test.shape[1]))

In [7]:
# Create and fit the LSTM model
model = create_model()
model.fit(X_train, train, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
928/928 - 13s - loss: 0.0184 - 13s/epoch - 14ms/step
Epoch 2/100
928/928 - 5s - loss: 0.0011 - 5s/epoch - 5ms/step
Epoch 3/100
928/928 - 5s - loss: 9.0461e-04 - 5s/epoch - 5ms/step
Epoch 4/100
928/928 - 5s - loss: 8.6567e-04 - 5s/epoch - 5ms/step
Epoch 5/100
928/928 - 5s - loss: 7.6659e-04 - 5s/epoch - 5ms/step
Epoch 6/100
928/928 - 5s - loss: 6.7420e-04 - 5s/epoch - 5ms/step
Epoch 7/100
928/928 - 5s - loss: 5.2631e-04 - 5s/epoch - 5ms/step
Epoch 8/100
928/928 - 5s - loss: 5.5633e-04 - 5s/epoch - 5ms/step
Epoch 9/100
928/928 - 5s - loss: 4.2058e-04 - 5s/epoch - 5ms/step
Epoch 10/100
928/928 - 5s - loss: 4.0348e-04 - 5s/epoch - 5ms/step
Epoch 11/100
928/928 - 5s - loss: 3.8284e-04 - 5s/epoch - 5ms/step
Epoch 12/100
928/928 - 5s - loss: 3.7007e-04 - 5s/epoch - 5ms/step
Epoch 13/100
928/928 - 5s - loss: 3.3845e-04 - 5s/epoch - 5ms/step
Epoch 14/100
928/928 - 5s - loss: 3.7970e-04 - 5s/epoch - 5ms/step
Epoch 15/100
928/928 - 5s - loss: 3.4439e-04 - 5s/epoch - 5ms/step
Epoch 16/

In [8]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

15/15 [==============================] - 1s 5ms/step


In [9]:
train_predict = scaler.inverse_transform(train_predict)
train = scaler.inverse_transform(train)
test_predict = scaler.inverse_transform(test_predict)
test = scaler.inverse_transform(test)

In [10]:
"""train_rmse = np.sqrt(np.mean(np.power((train - train_predict), 2)))
test_rmse = np.sqrt(np.mean(np.power((test - test_predict), 2)))
print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)"""
from sklearn.metrics import r2_score
import math
from sklearn.metrics import mean_squared_error
train_rmse = np.sqrt(mean_squared_error(train, train_predict))
test_rmse = np.sqrt(mean_squared_error(test, test_predict))
print('R2 Score_train:', r2_score(train, train_predict))
print('R2 Score_test:', r2_score(test, test_predict))

print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)

R2 Score_train: 0.9999095902333127
R2 Score_test: 0.9998682371051859
Train RMSE: 0.45
Test RMSE: 0.59


In [11]:
df=pd.DataFrame({'Actual':train.tolist(),'Predicted from lstm':train_predict.tolist()})
df.to_excel('lstm_10.xlsx')
df

,Actual,Predicted from lstm
0,[91.6568832397461],[91.29685974121094]
1,[91.6568832397461],[91.29685974121094]
2,[91.6568832397461],[91.29685974121094]
3,[91.6568832397461],[91.29685974121094]
4,[91.6568832397461],[91.29685974121094]
...,...,...
923,[162.0800018310547],[161.4417724609375]
924,[168.11000061035156],[167.3939666748047]
925,[146.8699951171875],[146.36825561523438]
926,[164.42999267578125],[163.76321411132812]


In [12]:
df=pd.DataFrame({'Actual':test.tolist(),'Predicted from lstm':test_predict.tolist()})
df.to_excel('lstm_10_test.xlsx')
df

,Actual,Predicted from lstm
0,[208.42999267578125],[206.7145233154297]
1,[208.91000366210938],[207.17657470703125]
2,[184.97000122070312],[183.9477996826172]
3,[189.22000122070312],[188.09719848632812]
4,[195.32000732421875],[194.0343780517578]
...,...,...
453,[64.73999786376953],[64.51795959472656]
454,[75.4800033569336],[75.18185424804688]
455,[70.19999694824219],[69.93498992919922]
456,[47.11000061035156],[47.101280212402344]
